# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [43]:
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core import Model

import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initializing the workspace

In [20]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'fraud-detection'

experiment=Experiment(ws, experiment_name)

## Creating the compute

In [5]:
try:
    compute_target = ComputeTarget(workspace=ws, name='cluster')
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, 'cluster', compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
#Execute this locally on git shell and upload the dataset
! pip install kaggle
! kaggle datasets download -d mlg-ulb/creditcardfraud
! unzip creditcardfraud.zip

 86%|████████████████████████████████▊     | 57.0M/66.0M [00:00<00:00, 71.4MB/s]
100%|██████████████████████████████████████| 66.0M/66.0M [00:01<00:00, 47.0MB/s]


In [22]:
found = False
key = "creditcardfraud"
description_text = "Dataset for capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        print("Please execute above commands and create a dataset")

df = dataset.to_pandas_dataframe()
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284806.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.841366,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.958151e-16,-3.147640e-15,...,1.471982e-16,8.042109e-16,5.282450e-16,4.458267e-15,1.426896e-15,1.701640e-15,-3.671606e-16,-1.218152e-16,88.349619,0.001727
std,47488.228330,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.250000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84691.500000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.750000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [27]:
automl_settings = {
    "experiment_timeout_minutes":60, # To make sure the experiment finishes on time before the timeout as per lab instructions
    "max_concurrent_iterations": 5, # To reach highest level of concurrency as per cluster configuration
    "primary_metric" : 'AUC_weighted' # To identify the score of each model that AutoML trains
}
automl_config = AutoMLConfig(compute_target=compute_target, # To specify out compute target to run the experiment on
                             task = "classification", # We are identifying if the transaction is fraudulant or not
                             training_data=dataset, # Location of the dataset
                             label_column_name="Class", # Column name of the target variable
#                              path = project_folder,
                             enable_early_stopping= True, # To save the computation cost
                             featurization= 'auto', # So that our AutoMl performs best preprocessing to create features
                             debug_log = "automl_errors.log", # To store logs
                             **automl_settings
                            )

In [28]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [30]:
remote_run.wait_for_completion(show_output=True)

{'runId': 'AutoML_8132042c-c606-4d2c-ae6a-f50d70b741ca',
 'target': 'cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-31T11:12:41.777621Z',
 'endTimeUtc': '2021-01-31T11:40:59.002424Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"fraud-detection","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-136702","workspace_name":"quick-starts-ws-136702","region":"southcentralus","compute_target":"cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [35]:
# Retrieve and save your best automl model.
import joblib

best, model = remote_run.get_output()
joblib.dump(model, './bestAutoMlModel.joblib')

['./bestAutoMlModel.joblib']

In [37]:
model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('4',
                                             Pipeline(memory=None,
                                                      steps=[('minmaxscaler',
                                                              MinMaxScaler(copy=True,
                                                                           feature_range=(0,
                                                                                          1))),
                                                          

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [39]:
#We dont need to choose the best model because it chooses best model default metric
model = remote_run.register_model(description = "AutoML Credit Card Fraud detection Best Model",
                               tags={'area': 'banks'}) 
print(remote_run.model_id)

AutoML8132042cc38


In [41]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-136702', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-136702'), name=AutoML8132042cc38, id=AutoML8132042cc38:1, version=1, tags={'area': 'banks'}, properties={})

In [55]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

env = best.get_environment()
best.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score.py')

inference_config = InferenceConfig(entry_script='outputs/score.py', environment=env)

In [56]:
service_name = 'credit-card-fraud-service'

service = Model.deploy(ws, service_name, [model], inference_config=inference_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [59]:
print(service.state)
print(service.scoring_uri)

Healthy
http://ed736b8b-3d1c-4f60-9deb-cc0af10a0d47.southcentralus.azurecontainer.io/score


In [168]:
test_fraud = df[df.Class==1].sample(2)
test_non_fraud = df.sample(3)
test = pd.concat([test_fraud,test_non_fraud])
actual = test.Class
del test["Class"]
test_json = json.dumps({"data":test.to_dict(orient="records")})
print(test_json)

{"data": [{"Time": 55614.0, "V1": -7.34795494690262, "V2": 2.39704134824746, "V3": -7.57235581757844, "V4": 5.17781925521834, "V5": -2.85483828261718, "V6": -1.79523877381611, "V7": -8.78323473517424, "V8": 0.437156561210431, "V9": -3.74059750992972, "V10": -8.33286306292127, "V11": 5.76318940883843, "V12": -8.70787929219866, "V13": -1.71694948774786, "V14": -9.57719439289905, "V15": 0.146369002626258, "V16": -7.58649078643175, "V17": -12.5039309695675, "V18": -4.37563101237668, "V19": 2.46519487038601, "V20": 0.0731637913502985, "V21": -0.175273369741836, "V22": 0.543325051876651, "V23": -0.547955067054245, "V24": -0.503721690122004, "V25": -0.31093277648824, "V26": -0.16398601754964, "V27": 1.19789504657005, "V28": 0.378186688214491, "Amount": 0.83}, {"Time": 7543.0, "V1": 0.329594333318222, "V2": 3.71288929524103, "V3": -5.77593510831666, "V4": 6.07826550560828, "V5": 1.66735901311948, "V6": -2.42016841351562, "V7": -0.812891249491333, "V8": 0.133080117970748, "V9": -2.2143113120496

In [170]:
print("Predicted Values", service.run(test_json))
print("Actual Values", actual.to_list())

Predicted Values {"result": [1, 1, 0, 0, 0]}
Actual Values [1, 1, 0, 0, 0]


TODO: In the cell below, print the logs of the web service and delete the service

In [171]:
service.get_logs()

'2021-01-31T12:28:42,899001050+00:00 - iot-server/run \n2021-01-31T12:28:43,062915686+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-01-31T12:28:43,162434022+00:00 - gunicorn/run \n2021-01-31T12:28:43,161657413+00:00 - rsyslog/run 

In [172]:
service.delete()